In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')
data = pd.read_csv('train.csv')
train = data.sample(frac=0.7, random_state=1)
test = data[data.index.isin(train.index)==False]
#train['Obs']=range(1,805)  # maybe change index from 1 to 804
train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,RL,NaN,6853,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2009,WD,220000
791,160,FV,30.0,3180,Pave,Pave,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,151000
571,20,RL,60.0,6960,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,Shed,500,11,2009,WD,120500
224,20,RL,70.0,8050,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,3,2007,WD,127000
498,20,RL,NaN,9556,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,8,2007,WD,160000


In [163]:
rows, columns = train.shape
print(rows)
print(columns)

563
79


In [164]:
train.describe().round(2) # without object type

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemod/Add,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,563.00,451.00,563.00,563.00,563.00,563.00,563.00,560.00,563.00,563.00,...,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00
mean,56.86,68.29,9800.64,5.97,5.63,1970.00,1982.61,103.04,439.10,54.17,...,90.55,41.44,25.37,1.73,14.31,1.02,21.76,6.06,2007.91,173867.17
std,41.65,22.60,5832.44,1.32,1.08,27.55,20.78,189.73,410.48,186.25,...,121.59,59.17,64.31,17.14,49.66,24.28,223.55,2.58,1.30,69561.87
min,20.00,21.00,1300.00,2.00,3.00,1880.00,1950.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2006.00,35000.00
25%,20.00,56.00,7490.00,5.00,5.00,1953.00,1963.50,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,2007.00,129000.00
50%,50.00,69.00,9259.00,6.00,5.00,1970.00,1990.00,0.00,404.00,0.00,...,0.00,16.00,0.00,0.00,0.00,0.00,0.00,6.00,2008.00,156000.00
75%,70.00,80.00,11366.50,7.00,6.00,1997.00,2002.00,148.00,695.50,0.00,...,166.00,62.00,0.00,0.00,0.00,0.00,0.00,7.00,2009.00,206500.00
max,190.00,155.00,70761.00,10.00,9.00,2008.00,2009.00,1224.00,1965.00,1526.00,...,646.00,365.00,364.00,219.00,287.00,576.00,4500.00,12.00,2010.00,538000.00


In [165]:
# variable preparation

predictors=list(train.columns[2:-1])
continuous_predictors=train.select_dtypes(['int64']).columns
columns_categorical=train.select_dtypes(['object']).columns


## Step 1: Categorial variables clean and discussion

In [166]:
train['MSZoning'].unique()

array(['RL', 'FV', 'RM', 'RH', 'C (all)', 'I (all)'], dtype=object)

In [167]:
# these variables are part of 'columns_categorical', 
categorical=['MSZoning', 'Street','LotShape', 'LandContour','Utilities','LotConfig', 'LandSlope'
           , 'Neighborhood', 'Condition1', 'Condition2' ,'BldgType','HouseStyle', 'RoofStyle', 'RoofMatl', 
             'Exterior1st','Exterior2nd','ExterQual', 'ExterCond', 'Foundation','Heating'
            , 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual','Functional'
            , 'PavedDrive','SaleType']


train['MSZoning']=train['MSZoning'].str.replace('(',"")
train['MSZoning']=train['MSZoning'].str.replace(')',"")
for item1 in categorical:
    train[item1]=train[item1].str.replace(' ',"")
    length=len(train[item1].unique())
    count=0
    while (length-1)>=count:
        for item2 in train[item1].unique():
            count1=str(count)
            train[item1]=train[item1].str.replace(item2,count1)
            count=int(count1)+1

In [168]:
#This is to replace NaN with empty string for now
train = train.replace(np.nan, '', regex=True)

#Use this to replace whatever first argument is with second argument
#train['Alley'] = train['Alley'].replace(0,'' , regex=True)


### Only 3 values with pool variables, we can just drop this

In [169]:
del train['PoolQC']
del train['PoolArea']
del test['PoolQC']
del test['PoolArea']

In [170]:
df = pd.DataFrame({'Fence':list(train['Fence'])})
pd.get_dummies(df).head()
#Here, the base case Fence means the house does not have a 

,Fence_,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


23 places have a feature out of 805 observations. Is it worthwhile to even include? Only 2% of observations have this

Therefore, I decided to remove it.

In [171]:
del train['MiscFeature']
del test['MiscFeature']

In [172]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,0,,6853,0,,0,0,0,0,...,25,0,0,0,,0,6,2009,0,220000
791,160,1,30,3180,0,Pave,1,0,0,0,...,166,0,0,0,,0,6,2006,0,151000
571,20,0,60,6960,0,,1,0,0,0,...,0,0,0,0,,500,11,2009,0,120500
224,20,0,70,8050,0,,1,0,0,0,...,64,0,0,0,,0,3,2007,0,127000
498,20,0,,9556,0,,0,0,0,1,...,21,0,0,0,,0,8,2007,0,160000


In [173]:
del train['Alley']
del test['Alley']

In [174]:
#Need to conver any string into integer for machine learning algorithm to work
combined_set = pd.concat([train, test], axis = 0)

In [175]:
for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categorical strings
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Replace strings with an integer

In [176]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 8 to 802
Data columns (total 75 columns):
MSSubClass       804 non-null int64
MSZoning         804 non-null int8
LotFrontage      804 non-null int8
LotArea          804 non-null int64
Street           804 non-null int8
LotShape         804 non-null int8
LandContour      804 non-null int8
Utilities        804 non-null int8
LotConfig        804 non-null int8
LandSlope        804 non-null int8
Neighborhood     804 non-null int8
Condition1       804 non-null int8
Condition2       804 non-null int8
BldgType         804 non-null int8
HouseStyle       804 non-null int8
OverallQual      804 non-null int64
OverallCond      804 non-null int64
YearBuilt        804 non-null int64
YearRemod/Add    804 non-null int64
RoofStyle        804 non-null int8
RoofMatl         804 non-null int8
Exterior1st      804 non-null int8
Exterior2nd      804 non-null int8
MasVnrType       804 non-null int8
MasVnrArea       804 non-null int16
ExterQual    

In [177]:
final_train = combined_set[:train.shape[0]] # Up to the last initial training set row
final_test = combined_set[train.shape[0]:] # Past the last initial training set row

In [178]:
#This is the price variable we need to remove
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')

In [179]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# For this section, make sure to change the input_dim to current number of variables

In [121]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(77, input_dim=77, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [122]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=50, batch_size=5, verbose=0)kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, final_train, y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [118]:
# evaluate model with standardized dataset
#https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
#Deep learning model
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, final_train, y_train, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: 27916815861.29 (2900112774.80) MSE


In [124]:
#Now we develop a 2 layer hidden network

In [127]:
# define a new model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(77, input_dim=77, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [128]:
# evaluate model with standardized dataset
#https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
#Deep learning model
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, final_train, y_train, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: 3723519590.61 (704113682.96) MSE


# More neurons now on each level

In [134]:
# define wider model so now we have more neurons on each level by doubling it
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(77, input_dim=77, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [133]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, final_train, y_train, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: 15894078157.03 (1407353508.29) MSE


hi
